In [1]:
from pathlib import Path
import pandas as pd
from PIL import Image

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models

# 路径设置（和 01 一样）
ROOT_DIR = Path.cwd().parent   # 确保这个位置和 01/003 一致
DATA_DIR = ROOT_DIR / "data"
CSV      = DATA_DIR / "products.csv"

df = pd.read_csv(CSV)
print("rows, cols:", df.shape)
print(df.head(3))

# 只用 train/val 来训练（你也可以只用 train，看你）
df_train = df[df["eval_status"].isin(["train", "val"])].copy()

# 防止缺失或损坏的数据
df_train = df_train.dropna(subset=["image_path", "category"]).reset_index(drop=True)
print("df_train shape:", df_train.shape)


rows, cols: (289222, 5)
   product_id                                         image_path category  \
0           1  img/WOMEN/Blouses_Shirts/Sheer_Pleated-Front_B...   Blouse   
1           2  img/WOMEN/Blouses_Shirts/Sheer_Pleated-Front_B...   Blouse   
2           3  img/WOMEN/Blouses_Shirts/Sheer_Pleated-Front_B...   Blouse   

   is_in_stock eval_status  
0            1       train  
1            1       train  
2            1         val  
df_train shape: (249222, 5)


In [2]:
# 给类别做一个 label 编码：比如 'Blouse' -> 0, 'Dress' -> 1, ...
cat2idx = {c: i for i, c in enumerate(sorted(df_train["category"].unique()))}
idx2cat = {v: k for k, v in cat2idx.items()}
num_classes = len(cat2idx)
print("num_classes:", num_classes)

# 图像增强 / 预处理（跟你 003 里的差不多）
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])

class FashionDataset(Dataset):
    def __init__(self, df, data_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.data_dir = data_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.data_dir / row["image_path"]
        
        # 打开图像
        img = Image.open(img_path).convert("RGB")
        
        if self.transform is not None:
            img = self.transform(img)
        
        label_str = row["category"]
        label = cat2idx[label_str]
        
        return img, label

dataset = FashionDataset(df_train, DATA_DIR, transform=transform)
print("Dataset size:", len(dataset))


num_classes: 46
Dataset size: 249222


In [3]:
from sklearn.model_selection import train_test_split

train_idx, val_idx = train_test_split(
    range(len(dataset)), test_size=0.2, random_state=42, shuffle=True
)

from torch.utils.data import Subset

train_ds = Subset(dataset, train_idx)
val_ds   = Subset(dataset, val_idx)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=0)

print("train batches:", len(train_loader), "val batches:", len(val_loader))


train batches: 6231 val batches: 1558


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

# 1）加载 ImageNet 预训练 resnet18
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# 2）改最后一层全连接输出维度为 num_classes
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


device: cpu


In [5]:
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    loss_sum = 0.0
    with torch.no_grad():
        for xb, yb in dataloader:
            xb = xb.to(device)
            yb = yb.to(device)
            logits = model(xb)
            loss = criterion(logits, yb)
            loss_sum += loss.item() * xb.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == yb).sum().item()
            total += xb.size(0)
    return loss_sum / total, correct / total

num_epochs = 3  # 先来 3 轮，如果可以再加

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        xb = xb.to(device)
        yb = yb.to(device)

        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * xb.size(0)

    train_loss = running_loss / len(train_ds)
    val_loss, val_acc = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - train_loss: {train_loss:.4f}, "
          f"val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f}")


FileNotFoundError: [Errno 2] No such file or directory: 'd:\\Icey\\tcd\\notebooks\\dMining\\Group Project\\workflow\\data\\img\\WOMEN\\Blouses_Shirts\\Daisy_Graphic_Muscle_Tee\\img_00000008.jpg'